In [18]:
import os
import pandas as pd
import numpy as np
from hyperopt import hp, fmin, rand, tpe, space_eval,Trials,STATUS_OK
from sklearn.model_selection import cross_val_score

In [19]:

factors = ['Beta60', 'OperatingRevenueGrowRate', 'NetProfitGrowRate', 'NetCashFlowGrowRate', 'NetProfitGrowRate5Y', 'TVSTD20',
           'TVSTD6', 'TVMA20', 'TVMA6', 'BLEV', 'MLEV', 'CashToCurrentLiability', 'CurrentRatio', 'REC', 'DAREC', 'GREC',
           'DASREV', 'SFY12P', 'LCAP', 'ASSI', 'LFLO', 'TA2EV', 'PEG5Y', 'PE', 'PB', 'PS', 'SalesCostRatio', 'PCF', 'CETOP',
           'TotalProfitGrowRate', 'CTOP', 'MACD', 'DEA', 'DIFF', 'RSI', 'PSY', 'BIAS10', 'ROE', 'ROA', 'ROA5', 'ROE5',
           'DEGM', 'GrossIncomeRatio', 'ROECut', 'NIAPCut', 'CurrentAssetsTRate', 'FixedAssetsTRate', 'FCFF', 'FCFE', 'PLRC6',
           'REVS5', 'REVS10', 'REVS20', 'REVS60', 'HSIGMA', 'HsigmaCNE5', 'ChaikinOscillator', 'ChaikinVolatility', 'Aroon',
           'DDI', 'MTM', 'MTMMA', 'VOL10', 'VOL20', 'VOL5', 'VOL60', 'RealizedVolatility', 'DASTD', 'DDNSR', 'Hurst']

df = pd.read_csv('dataset_1.csv',encoding='GBK')
df.head()

,Unnamed: 0,ticker,tradeDate,Beta60,OperatingRevenueGrowRate,NetProfitGrowRate,NetCashFlowGrowRate,NetProfitGrowRate5Y,TVSTD20,TVSTD6,...,RealizedVolatility,DASTD,DDNSR,Hurst,next_month_end,abs_return,active_return,industryName1,label,year
0,0,1,20070131,-0.838573,-0.527473,2.064908,-0.235434,-0.120729,0.478006,0.268336,...,0.0,-0.603847,-1.160788,0.0,20070228,-0.004221,-0.071021,银行,-1,2007
1,1,2,20070131,0.603209,0.476466,-0.019215,-0.013746,-1.210592,0.762963,0.696981,...,0.0,0.655525,-0.847547,0.0,20070228,-0.037359,-0.104159,房地产,-1,2007
2,2,60,20070131,1.439494,-0.559091,1.508711,0.085016,1.423780,2.340842,1.987375,...,0.0,0.747250,0.638908,0.0,20070228,0.171481,0.104681,有色金属,0,2007
3,3,63,20070131,-0.221436,-0.891169,-0.690564,1.331827,0.033096,1.378065,0.707162,...,0.0,-0.694765,-0.580936,0.0,20070228,0.093903,0.027103,通信,0,2007
4,4,69,20070131,-0.882542,0.101637,-0.671414,0.191923,0.463894,2.255643,1.680278,...,0.0,0.640970,0.450284,0.0,20070228,0.020656,-0.046144,房地产,-1,2007


In [35]:
train_df = df[df['year']<2012]
train_df = train_df[train_df['label']!= 0]
train_y = train_df.loc[:,'label'].apply(lambda x:0 if x == -1 else x)
train_y = train_y.values.copy()
train_x = train_df[factors].values.copy()

In [36]:
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
def my_custom_score(ground_truth,prediction):
    return roc_auc_score(ground_truth,prediction)

my_score = make_scorer(my_custom_score,greater_is_better =True)

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
def obj(params,x=train_x,y=train_y):
    rf = RandomForestClassifier(max_depth = params['max_depth'],max_features = params['max_features'],n_estimators = params['n_estimators'],\
                                min_samples_split = params['min_samples_split'], min_samples_leaf = params['min_samples_leaf'],n_jobs = 8)
    scores = cross_val_score(rf,x,y,cv = 3,scoring = my_score)
    
    print('params:',params)
    print('scores:',scores)
    print('scores_average:',np.average(scores))
    
    return {'loss': np.average(scores),'status':STATUS_OK}


In [38]:
space = {
    'max_depth': hp.choice('max_depth',list(range(5,20))),
    'max_features' : hp.choice('max_features',list(range(10,50))),
    'n_estimators':hp.choice('n_estimators',list(range(40,100))),
    'min_samples_split':hp.choice('min_samples_split',list(range(30,70))),
    'min_samples_leaf':hp.choice('min_samples_leaf',list(range(5,40)))
}
trials = Trials()
best = fmin(obj,space,algo=tpe.suggest,max_evals = 100,trials = trials)
print(space_eval(space,best))

params: {'max_depth': 19, 'max_features': 10, 'min_samples_leaf': 26, 'min_samples_split': 32, 'n_estimators': 83}
scores: [ 0.54103404  0.54197373  0.53589536]
scores_average: 0.539634376704
params: {'max_depth': 9, 'max_features': 18, 'min_samples_leaf': 28, 'min_samples_split': 57, 'n_estimators': 87}
scores: [ 0.53864309  0.54772621  0.54134108]
scores_average: 0.542570125984
params: {'max_depth': 9, 'max_features': 33, 'min_samples_leaf': 31, 'min_samples_split': 67, 'n_estimators': 53}
scores: [ 0.54814795  0.55016008  0.53757093]
scores_average: 0.545292985762
params: {'max_depth': 16, 'max_features': 37, 'min_samples_leaf': 6, 'min_samples_split': 59, 'n_estimators': 80}
scores: [ 0.54028531  0.54830858  0.5351439 ]
scores_average: 0.541245928982
params: {'max_depth': 7, 'max_features': 45, 'min_samples_leaf': 39, 'min_samples_split': 37, 'n_estimators': 42}
scores: [ 0.53820796  0.54637818  0.53305287]
scores_average: 0.539213001262
params: {'max_depth': 11, 'max_features': 48

params: {'max_depth': 9, 'max_features': 30, 'min_samples_leaf': 16, 'min_samples_split': 57, 'n_estimators': 89}
scores: [ 0.54543548  0.54773305  0.53786184]
scores_average: 0.543676786966
params: {'max_depth': 9, 'max_features': 46, 'min_samples_leaf': 29, 'min_samples_split': 34, 'n_estimators': 98}
scores: [ 0.53924267  0.55119889  0.53530725]
scores_average: 0.541916272776
params: {'max_depth': 18, 'max_features': 12, 'min_samples_leaf': 31, 'min_samples_split': 31, 'n_estimators': 91}
scores: [ 0.54359901  0.5445387   0.53468498]
scores_average: 0.540940898716
params: {'max_depth': 7, 'max_features': 15, 'min_samples_leaf': 39, 'min_samples_split': 34, 'n_estimators': 58}
scores: [ 0.54349551  0.54878963  0.5377269 ]
scores_average: 0.543337348168
params: {'max_depth': 6, 'max_features': 34, 'min_samples_leaf': 24, 'min_samples_split': 64, 'n_estimators': 71}
scores: [ 0.53896625  0.54171644  0.53862204]
scores_average: 0.539768245025
params: {'max_depth': 9, 'max_features': 40,

params: {'max_depth': 7, 'max_features': 13, 'min_samples_leaf': 9, 'min_samples_split': 30, 'n_estimators': 83}
scores: [ 0.5439342   0.54773933  0.53652856]
scores_average: 0.542734029643
params: {'max_depth': 8, 'max_features': 16, 'min_samples_leaf': 5, 'min_samples_split': 55, 'n_estimators': 99}
scores: [ 0.5440817   0.54380309  0.53968983]
scores_average: 0.542524875577
params: {'max_depth': 18, 'max_features': 47, 'min_samples_leaf': 13, 'min_samples_split': 36, 'n_estimators': 46}
scores: [ 0.54134463  0.54378068  0.53031635]
scores_average: 0.538480552043
params: {'max_depth': 11, 'max_features': 19, 'min_samples_leaf': 31, 'min_samples_split': 54, 'n_estimators': 81}
scores: [ 0.54497438  0.5498361   0.53530261]
scores_average: 0.543371027834
params: {'max_depth': 14, 'max_features': 48, 'min_samples_leaf': 7, 'min_samples_split': 42, 'n_estimators': 86}
scores: [ 0.54346435  0.54272519  0.53092249]
scores_average: 0.539037344645
params: {'max_depth': 19, 'max_features': 27,